# Домашнее задание 3

## ЗАДАЧА
В приложенных файлах вы увидите данные по прохождению студентами тестов на образовательной платформе. Такие данные могут быть использованы для построения персонализированных ИИ-ассистентов для поддержки обучения студента. На текущем этапе сосредоточимся на начальном этапе — сборе и подготовке данных.

## ВАМ НЕОБХОДИМО
Используя библиотеку pandas (или аналоги), проанализировать как можно больше характеристик, влияющих на успеваемость студентов.

## ОПИСАНИЕ ДАННЫХ
Для выполнения задания вам необходимы файлы train.csv, lectures.csv, questions.csv.

TRAIN.CSV
* row_id: (int64) ID code for the row.
* timestamp: (int64) the time in milliseconds between this user interaction and the first event completion from that user.
* user_id: (int32) ID code for the user.
* content_id: (int16) ID code for the user interaction
* content_type_id: (int8) 0 if the event was a question being posed to the user, 1 if the event was the user watching a lecture.
* task_container_id: (int16) Id code for the batch of questions or lectures. For example, a user might see three questions in a row before seeing the explanations for any of them. Those three would all share a task_container_id.
* user_answer: (int8) the user's answer to the question, if any. Read -1 as null, for lectures.
* answered_correctly: (int8) if the user responded correctly. Read -1 as null, for lectures.
* prior_question_elapsed_time: (float32) The average time in milliseconds it took a user to answer each question in the previous question bundle, ignoring any lectures in between. Is null for a user's first question bundle or lecture. Note that the time is the average time a user took to solve each question in the previous bundle.
* prior_question_had_explanation: (bool) Whether or not the user saw an explanation and the correct response(s) after answering the previous question bundle, ignoring any lectures in between. The value is shared across a single question bundle, and is null for a user's first question bundle or lecture. Typically the first several questions a user sees were part of an onboarding diagnostic test where they did not get any feedback.
 

QUESTIONS.CSV: METADATA FOR THE QUESTIONS POSED TO USERS.
* question_id: foreign key for the train/test content_id column, when the content type is question (0).
* bundle_id: code for which questions are served together.
* correct_answer: the answer to the question. Can be compared with the train user_answer column to check if the user was right.
* part: the relevant section of the TOEIC test.
* tags: one or more detailed tag codes for the question. The meaning of the tags will not be provided, but these codes are sufficient for clustering the questions together.
 

LECTURES.CSV: METADATA FOR THE LECTURES WATCHED BY USERS AS THEY PROGRESS IN THEIR EDUCATION.
* lecture_id: foreign key for the train/test content_id column, when the content type is lecture (1).
* part: top level category code for the lecture.
* tag: one tag codes for the lecture. The meaning of the tags will not be provided, but these codes are sufficient for clustering the lectures together.
* type_of: brief description of the core purpose of the lecture.

## Шесть шагов для создания успешной модели машинного обучения
* Уточните назначение модели.
* Задайте необходимые критерии.
* Идеальные новые функции.
* Стройте функции как входные данные.
* Изучите влияние.
* Уточните функции.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [2]:
start = time.time()
df_train = pd.read_csv('data/train.csv')
end = time.time()
print(end-start)

49.90296697616577


Вывод: датасет очень большой и загружается в память почти 50 сек

In [3]:
df_train.head()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,5692,0,1,3,1,NaN,NaN
1,1,56943,115,5716,0,2,2,1,37000.0,False
2,2,118363,115,128,0,0,0,1,55000.0,False
3,3,131167,115,7860,0,3,0,1,19000.0,False
4,4,137965,115,7922,0,4,1,1,11000.0,False


In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101230332 entries, 0 to 101230331
Data columns (total 10 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   row_id                          int64  
 1   timestamp                       int64  
 2   user_id                         int64  
 3   content_id                      int64  
 4   content_type_id                 int64  
 5   task_container_id               int64  
 6   user_answer                     int64  
 7   answered_correctly              int64  
 8   prior_question_elapsed_time     float64
 9   prior_question_had_explanation  object 
dtypes: float64(1), int64(8), object(1)
memory usage: 7.5+ GB


Вывод: датасет содержит 101 млн наблюдений и занимает в памяти 7.5 Гб. Можно попопробовать оптимизировать датасет проверив количество уникальных значений в фичах (признаках).

In [14]:
print(df_train['user_id'].nunique())
print(df_train['user_id'].max())

393656
2147482888


In [15]:
print(df_train['content_id'].nunique())
print(df_train['content_id'].max())

13782
32736


In [16]:
print(df_train['content_type_id'].nunique())
print(df_train['content_type_id'].max())

2
1


In [17]:
print(df_train['task_container_id'].nunique())
print(df_train['task_container_id'].max())

10000
9999


In [18]:
print(df_train['user_answer'].nunique())
print(df_train['user_answer'].max())

5
3


In [19]:
print(df_train['answered_correctly'].nunique())
print(df_train['answered_correctly'].max())

3
1


In [20]:
print(df_train['prior_question_had_explanation'].nunique())

2


Столбец row_id является индексом строк и его можно удалить.
Фичи 2-7 являются int64 и поэтому занимают много места в памяти. Судя по разряду чисел и количеству уникальных значений возможны следующие преобразования:
1. user_id - int32
2. content_id - int16
3. content_type_id - int8
4. task_container_id - int16
5. user_answer - int8
6. answered_correctly - int8
7. prior_question_had_explanation - bool

## Оптимизируем датафрейм
Преобразуем типы данных для фичей

In [21]:
pd.options.display.float_format ='{:,.2f}'.format

In [22]:
df_train['user_id'] = df_train['user_id'].astype('int32')
df_train['content_id'] = df_train['content_id'].astype('int16')
df_train['content_type_id'] = df_train['content_type_id'].astype('int8')
df_train['task_container_id'] = df_train['task_container_id'].astype('int16')
df_train['user_answer'] = df_train['user_answer'].astype('int8')
df_train['answered_correctly'] = df_train['answered_correctly'].astype('int8')
df_train['prior_question_had_explanation'] = df_train['prior_question_had_explanation'].astype('bool')

Удаляем столбец row_id

In [25]:
df_train.drop('row_id', axis=1, inplace=True)

In [26]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101230332 entries, 0 to 101230331
Data columns (total 9 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   timestamp                       int64  
 1   user_id                         int32  
 2   content_id                      int16  
 3   content_type_id                 int8   
 4   task_container_id               int16  
 5   user_answer                     int8   
 6   answered_correctly              int8   
 7   prior_question_elapsed_time     float64
 8   prior_question_had_explanation  bool   
dtypes: bool(1), float64(1), int16(2), int32(1), int64(1), int8(3)
memory usage: 2.6 GB


In [27]:
df_train.head()

,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,115,5692,0,1,3,1,NaN,True
1,56943,115,5716,0,2,2,1,"37,000.00",False
2,118363,115,128,0,0,0,1,"55,000.00",False
3,131167,115,7860,0,3,0,1,"19,000.00",False
4,137965,115,7922,0,4,1,1,"11,000.00",False


Размер датасета в памяти сократился с 7.5 Гб до 2.6 Гб. Почти в 3 раза!

## Аналогично оптимизируем остальные датасеты

In [28]:
start = time.time()
df_questions = pd.read_csv('data/QUESTIONS.csv')
end = time.time()
print(end-start)

0.019655227661132812


In [29]:
df_questions.head()

,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92
3,3,3,0,1,131 149 162 29
4,4,4,3,1,131 5 162 38


In [30]:
df_questions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13523 entries, 0 to 13522
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   question_id     13523 non-null  int64 
 1   bundle_id       13523 non-null  int64 
 2   correct_answer  13523 non-null  int64 
 3   part            13523 non-null  int64 
 4   tags            13522 non-null  object
dtypes: int64(4), object(1)
memory usage: 528.4+ KB


In [31]:
print(df_questions['question_id'].nunique())
print(df_questions['question_id'].max())

13523
13522


In [32]:
print(df_questions['bundle_id'].nunique())
print(df_questions['bundle_id'].max())

9765
13522


In [33]:
print(df_questions['correct_answer'].nunique())
print(df_questions['correct_answer'].max())

4
3


In [34]:
print(df_questions['part'].nunique())
print(df_questions['part'].max())

7
7


In [35]:
df_questions['question_id'] = df_questions['question_id'].astype('int16')
df_questions['bundle_id'] = df_questions['bundle_id'].astype('int16')
df_questions['correct_answer'] = df_questions['correct_answer'].astype('int8')
df_questions['part'] = df_questions['part'].astype('int8')

In [36]:
df_questions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13523 entries, 0 to 13522
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   question_id     13523 non-null  int16 
 1   bundle_id       13523 non-null  int16 
 2   correct_answer  13523 non-null  int8  
 3   part            13523 non-null  int8  
 4   tags            13522 non-null  object
dtypes: int16(2), int8(2), object(1)
memory usage: 185.0+ KB


In [37]:
start = time.time()
df_lectures = pd.read_csv('data/LECTURES.csv')
end = time.time()
print(end-start)

0.010395050048828125


In [38]:
df_lectures.head()

,lecture_id,tag,part,type_of
0,89,159,5,concept
1,100,70,1,concept
2,185,45,6,concept
3,192,79,5,solving question
4,317,156,5,solving question


In [39]:
df_lectures.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   lecture_id  418 non-null    int64 
 1   tag         418 non-null    int64 
 2   part        418 non-null    int64 
 3   type_of     418 non-null    object
dtypes: int64(3), object(1)
memory usage: 13.2+ KB


In [40]:
print(df_lectures['lecture_id'].nunique())
print(df_lectures['lecture_id'].max())

418
32736


In [41]:
print(df_lectures['tag'].nunique())
print(df_lectures['tag'].max())

151
187


In [42]:
print(df_lectures['part'].nunique())
print(df_lectures['part'].max())

7
7


In [43]:
print(df_lectures['type_of'].nunique())

4


In [44]:
df_lectures['lecture_id'] = df_lectures['lecture_id'].astype('int16')
df_lectures['tag'] = df_lectures['tag'].astype('int16')
df_lectures['part'] = df_lectures['part'].astype('int8')

In [45]:
df_lectures.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   lecture_id  418 non-null    int16 
 1   tag         418 non-null    int16 
 2   part        418 non-null    int8  
 3   type_of     418 non-null    object
dtypes: int16(2), int8(1), object(1)
memory usage: 5.4+ KB
